<a href="https://colab.research.google.com/github/vshalisko/Phylogenetics/blob/master/Unidad_2/Chatbot_oraculo_ejemplo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### ¿Que tan facil es crear un Chatbot con Python?

> It will take exactly 4 months, 3 weeks, 2 days, 1 hour, 46 minutes and 15 seconds to learn everything you need to know to build a simple chatbot. (opinión de un usuario de Reddit) https://www.reddit.com/r/learnpython/comments/1dc80lz/making_simple_chatbot/

### Respuesta: Depende del tipo de Chatbot



Vamos a crear un Chatbot simple inspirado en código en un proyecto introductorio de Hyperskill
* https://hyperskill.org/projects/97

Soluciones:
* https://github.com/syyynth/hyperskill/tree/main/python/0097%20-%20Simple%20Chatty%20Bot%20(Python)
* https://github.com/Mathesh099/Simple-Chatty-Bot


Adicionalmente (en la etapa avanzada del proyecto) vamos a ampliar nuestro chatbot con los los componentes que usan la biblioteca NLTK, similar a un ejemplo disponible en:
* https://planetachatbot.com/construyendo-chatbot-simple-desde-cero-en-python-usando-nltk/



## Etapa básica del proyecto
### Los elementos incluidos en siguientes apartados son obligatorios para una entrega de la tarea

### Vamos a saludar el usuario

In [38]:
print('¡Hola! Soy un ChatBot.')
print('Me crearon en 2025 como parte de un ejercicio.')
print('¿Y tu nombre cual es?.')
nombre_usuario = str(input())
print('¡Me encanta tu nombre, ' + nombre_usuario + "!")

¡Hola! Soy un ChatBot.
Me crearon en 2025 como parte de un ejercicio.
¿Y tu nombre cual es?.
Slava
¡Me encanta tu nombre, Slava!


### Ahora vamos a adivinar edad del usuario. Para esto le vamos a hacer algunas preguntas

In [40]:
def adivinar_edad():
    print('Me gustaria adivinar tu edad, ' + nombre_usuario + '.')
    print('Para esto voy a hacer tres preguntas.')

    rem3 = int(input('¿Cual es el remanente de división de tu edad en 3?\n'))
    rem5 = int(input('¿Cual es el remanente de división de tu edad en 5?\n'))
    rem7 = int(input('¿Cual es el remanente de división de tu edad en 7?\n'))
    age = (rem3 * 70 + rem5 * 21 + rem7 * 15) % 105

    print("Creo que tu edad es " + str(age) + ". ¡Es un momento maravilloso para estudiar Python!")

adivinar_edad()

Me gustaria adivinar tu edad, Slava.
Para esto voy a hacer tres preguntas.
¿Cual es el remanente de división de tu edad en 3?
2
¿Cual es el remanente de división de tu edad en 5?
2
¿Cual es el remanente de división de tu edad en 7?
5
Creo que tu edad es 47. ¡Es un momento maravilloso para estudiar Python!


## Etapa avanzada del proyecto
### Todo lo que se presenta aqui es opcional, pero permite hacer nuestro chatbot más divertido

* Instalar la biblioteca **gensim** en el entorno virtual de Python.

* Nota: Al momento de instalar la biblioteca por primera vez surante la sesion se requiere permitir reiniciar la sesión de Colab.

In [20]:
%pip install gensim

* Cargar los componentes (bibliotecas) **re**, **nltk**, **numpy**, y algunos componentes de **gensim** y **sklearn**

In [21]:
import re
import nltk
import numpy as np

from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity

* Descargar las bases de datos de **nltk**

In [22]:
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


* Cargar las palabras de español incluidas en la lista de **stopwprds**

In [23]:
spanish_sw = set(stopwords.words('spanish'))
#print(spanish_sw)

* Conectar **Google Drive** del usuario (de alla vamos a cargar la base de datos de frases celebres)

In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


* Leer el archivo con la base de datos de frases celebres desde **Google Drive**

In [34]:
path = '/content/drive/MyDrive/Colab Data/'
file = 'frases_celebres_seleccion.txt'

frases = ''
with open(path + file, 'r', errors = 'ignore') as f:
  frases = f.read()
  #print(frases)

* Pre-procesar la base de datos de frases célebres
1. Convertir texto a registro bajo
2. Separar las frases
3. Separar palabras (opcional)

In [41]:
# convertir texto plano a lista de frases
onlysent_tokens = nltk.sent_tokenize(frases)
# reemplazar simbolos de nueva linea ocn espacios
onlysent_tokens = [sentence.replace('\n', ' ') for sentence in onlysent_tokens]

# generar una copia de texto con en lowercase
frases_lc = frases.lower()
# convertir texto a lista de palabras
word_tokens = nltk.word_tokenize(frases_lc)

print("\nNúmero de frases célebres en la base de datos:")
print(len(onlysent_tokens))

print("\nEjemplo de frases celebres:")
print(onlysent_tokens[0:3])

print("\nNúmero de palabras en labase de datos de frases célebres:")
print(len(word_tokens))



Número de frases célebres en la base de datos:
639

Ejemplo de frases celebres:
['¿Encontraría a la Maga?', 'Tan de vez en cuando uno tiene ganas de encontrarse a sí mismo.', 'A buen hambre, no hay pan duro.']

Número de palabras en labase de datos de frases célebres:
7329


* Construir el modelo **Word2Vec** a partir de la base de datos de frases célebres.

Trata de un modelo donde los grupos de palabras se colocan en un espacio de factores para representar el vector de su significado

In [42]:
# Preprocesar texto para generar el modelo Word2Vec
# quitar los simbolos de punctuación y convertir palabras al registro bajo
corpus_sentences = [re.sub(r'[^\w\s]', '', sentence.lower()).split() for sentence in onlysent_tokens]
#print(type(corpus_sentences))
#print(len(corpus_sentences))
#print(corpus_sentences[0:3])

# funcion para eliminar stopwords
def remove_stopwords(sentence_words, sw):
  filtered_words = [word for word in sentence_words if word not in sw]
  return filtered_words

# eliminar stopwords de la base de datos
filtered_sentences = []
for sentence in corpus_sentences:
  filtered_sentence = remove_stopwords(sentence, spanish_sw)
  filtered_sentences.append(filtered_sentence)

#print(type(filtered_sentences))
#print(len(filtered_sentences))
#print(filtered_sentences[0:3])

# Entrenar el modelo Word2Vec
model = Word2Vec(min_count=1,
                     window=6,
                     vector_size=300,
                     sample=6e-5,
                     alpha=0.03,
                     min_alpha=0.0007,
                     negative=20,
                     workers=4)

model.build_vocab(filtered_sentences, progress_per=1000)
model.train(filtered_sentences, total_examples=model.corpus_count, epochs=30, report_delay=1)

# Entrenar el modelo Word2Vec (simple)
#model = Word2Vec(sentences=filtered_sentences, vector_size=100, window=5, min_count=1, workers=4)

print("\nCaracteristicas del modelo Word2Vec")
print(model)


Caracteristicas del modelo Word2Vec
Word2Vec<vocab=1534, vector_size=300, alpha=0.03>


### Ahora podemos regresar a interactuar con el usuario en marco de nuestro ChatBot
* Preguntar al usuario por una frase

In [43]:
user_input = str(input(user_name + ", introduce su pregunta o algo que te gustaria discutir: "))

Slava, introduce su pregunta o algo que te gustaria discutir: ¿Cunatos años se requiere para completar los estudios en la universidad?


* Comparar la frase con la base de datos de frases celebres y buscar la frase celebre con el significado mas cercana a la frase proporcionada por el usuario

In [44]:

# Funcion para construir el vector de una frase
def get_sentence_vector(sentence, model):
    # separar frase en palabras
    words = sentence.split()

    # quitar simbolos de puntuacion
    words = re.sub(r'[^\w\s]', '', sentence.lower()).split()

    # determinar el vector de cada palabra en la frase en forma de lista
    word_vectors = [model.wv[word] for word in words if word in model.wv]

    # en caso que vector es vacio generar el vector de ceros y regresarlo
    if not word_vectors:
        return np.zeros(model.vector_size)

    # en caso que vector no es vacio generar un promedio y regresarlo
    return np.mean(word_vectors, axis=0)

# Procesar la pregunta del usuario para construir su vector
filtered_user_input = remove_stopwords(user_input.split(), spanish_sw)
filtered_user_input_ok = " ".join(filtered_user_input)
user_input_vector = get_sentence_vector(filtered_user_input_ok, model)

print("Pregunta de " + nombre_usuario + ": " + str(user_input))
#print("Pregunta del ususrio filtrada: " + str(filtered_user_input))
#print("Vector del modelo:")
#print(user_input_vector)

# Encontrar la frase mas cercana por el signiicado entre frases célebres
most_similar_sentence = ""
max_similarity = -1

for i, sentence in enumerate(onlysent_tokens):
    sentence_vector = get_sentence_vector(sentence, model)
    if np.linalg.norm(user_input_vector) != 0 and np.linalg.norm(sentence_vector) != 0:
        similarity = cosine_similarity([user_input_vector], [sentence_vector])[0][0]
        if similarity > max_similarity:
            max_similarity = similarity
            most_similar_sentence = onlysent_tokens[i]

# La frase de mayor similitud:
#print("Primera frase")
#print(most_similar_sentence)
#print("Segunda frase")
#print(most_similar_sentence_2)


if not most_similar_sentence:
  print(nombre_usuario + ", lamento, pero no tengo nada que responder en esta ocasión")
else:
  print("La respuesta es: " + most_similar_sentence)

Pregunta de Slava: ¿Cunatos años se requiere para completar los estudios en la universidad?
La respuesta es: No hay mal que dure cien años.
